# Santander Dev Week 2023 (ETL com Python)

In [26]:
# Armazenando o endereço da API
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraindo a lista de IDs de usuário do arquivo CSV.

In [27]:
import pandas as pd

df = pd.read_csv('pipeline_etl.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[2777, 2778, 2780]


In [28]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))


[
  {
    "id": 2777,
    "name": "User_fourth",
    "account": {
      "id": 2931,
      "number": "4444444-4",
      "agency": "0001",
      "balance": 0.0,
      "limit": 1000.0
    },
    "card": {
      "id": 2696,
      "number": "**** **** 4444 5678",
      "limit": 1000.0
    },
    "features": [
      {
        "id": 927,
        "icon": "string",
        "description": "string"
      }
    ],
    "news": [
      {
        "id": 5514,
        "icon": "string",
        "description": "string"
      },
      {
        "id": 5536,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Garanta seu futuro com investimentos! Transforme sonhos em realidade e construa independ\u00eancia financeira."
      },
      {
        "id": 5537,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Invista em Bitcoins! Por ser descentralizado, oferece 

## **T**ransform

Utilizando a API do OpenAI GPT-4 para gerar uma mensagem para cada usuário.

In [29]:
!pip install openai

In [30]:
# Armazenando a API KEY:
openai_api_key = 'secret... :D'

In [36]:
# Gerando as mensagens para cada usuário
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância de comprar Bitcoins, explicando que os governos não podem embargar Bitcoins da forma que podem embargar moedas convencionais. Já que sua riqueza não pode ser congelada, Bitcoin lhe dá a liberdade de fazer o que quiser com seu dinheiro (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Invista em Bitcoin! Independente do controle do governo, seu valor seguro e livre para usá-lo como quiser.
Bitcoins: Liberdade financeira sem embargos governamentais. Você controla sua riqueza.
Invista em Bitcoin, a moeda que governos não controlam. Sua riqueza a salvo. Liberdade é Bitcoin!


## **L**oad

Ao final, atualizar a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [37]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User User_fourth updated? True!
User User_fifth updated? True!
User User_seventh updated? True!
